In [1]:
#!pip install customtkinter
#!python.exe -m pip install --upgrade pip

In [2]:
#!pip install --upgrade numpy scipy ultralytics opencv-python

In [3]:
#!pip install "numpy<2.0"

In [12]:
import os
import threading

import customtkinter
from tkinter import filedialog
from PIL import Image
import numpy as np

import REALIZAR_PREDICCION


class Aplicacion(customtkinter.CTk):

    def __init__(self):
        super().__init__()
        self.configurar_interfaz()
        self.archivo_seleccionado = None
        self.hilo_prediccion = None

    def configurar_interfaz(self):
        self.title("🦷 RootScan AI - Detección Inteligente de Canales Radiculares")
        self.geometry("1600x750")
        customtkinter.set_appearance_mode("dark")
        customtkinter.set_default_color_theme("blue")
        self.configure(padx=30, pady=30)
        
        self.boton_cargar = customtkinter.CTkButton(self, text="📂 Cargar Imagen", command=self.cargar_archivo,
            font=("Arial", 16, "bold"), corner_radius=10, width=200)
        self.boton_cargar.grid(row=0, column=0, pady=(10, 10), sticky="n")
        
        self.boton_ejecutar = customtkinter.CTkButton(self, text="📏 Calcular Longitud", command=self.ejecutar_prediccion,
            font=("Arial", 16, "bold"), corner_radius=10, width=200)
        self.boton_ejecutar.grid(row=1, column=0, pady=(10, 30), sticky="n")
        
        self.boton_guardar = customtkinter.CTkButton(self, text="💾 Guardar Resultados", command=self.guardar_resultados,
            font=("Arial", 16, "bold"), corner_radius=10, width=200)
        self.boton_guardar.grid(row=3, column=0, pady=(10, 30), sticky="n")

        self.etiqueta_original = self.crear_panel_imagen("    Radiografía Original 🦷", 1)
        self.grid_columnconfigure(2, minsize=30)
        
        self.etiqueta_mascara = self.crear_panel_imagen("    Segmentación del Canal 🧩", 3)
        self.grid_columnconfigure(4, minsize=30)
        
        self.etiqueta_esqueleto = self.crear_panel_imagen("    Esqueleto del Canal 🕸️", 5)
        self.grid_columnconfigure(6, minsize=30)
        
        self.etiqueta_longitud = self.crear_panel_imagen("    Medición en Píxeles 📏", 7)

        self.etiqueta_resultado1 = customtkinter.CTkLabel(self, text="📐 Píxeles totales: ---", font=("Arial", 18, "bold"),
            text_color="lightblue", anchor="w",)
        self.etiqueta_resultado1.grid(row=2, column=1, columnspan=4, pady=(30, 0), sticky="w")
        
        self.etiqueta_resultado2 = customtkinter.CTkLabel(self,text="📐 Longitud estimada: ---", font=("Arial", 18, "bold"),
            text_color="lightblue", anchor="w",)
        self.etiqueta_resultado2.grid(row=3, column=1, columnspan=4, pady=(30, 0), sticky="w")

    def crear_panel_imagen(self, titulo, columna):
        etiqueta_titulo = customtkinter.CTkLabel(self, text=titulo, font=("Arial", 16, "bold"), text_color="#CCCCCC",
            anchor="center")
        etiqueta_titulo.grid(row=0, column=columna, pady=(0, 5))
        etiqueta_imagen = customtkinter.CTkLabel(self, text="")
        etiqueta_imagen.grid(row=1, column=columna, padx=10, pady=10)
        return etiqueta_imagen

    def cargar_archivo(self):
        ruta = filedialog.askopenfilename(title="Seleccionar imagen", filetypes=[("Imágenes", "*.png *.jpg *.jpeg *.bmp")],)
        if ruta:
            self.archivo_seleccionado = ruta
            print(f"📁 Archivo cargado: {ruta}")
            self.after(0, lambda: self.mostrar_imagen(ruta, self.etiqueta_original))

    def mostrar_imagen(self, fuente, widget_etiqueta):
        if isinstance(fuente, str):
            imagen = Image.open(fuente)
        else:
            imagen = Image.fromarray(fuente.astype(np.uint8))
        def actualizar():
            dimensiones = (250, 400) if imagen.size[0] >= 720 else imagen.size
            imagen_tk = customtkinter.CTkImage(light_image=imagen, dark_image=imagen, size=dimensiones)
            widget_etiqueta.configure(image=imagen_tk, text="")
            widget_etiqueta.image = imagen_tk
        self.after(0, actualizar)

    def ejecutar_prediccion(self):
        if not self.archivo_seleccionado:
            print("⚠️ No se ha seleccionado ningún archivo")
            return
        if self.hilo_prediccion and self.hilo_prediccion.is_alive():
            print("⏳ Predicción en curso, espera a que termine.")
            return
        self.hilo_prediccion = threading.Thread(target=self.realizar_prediccion, daemon=True)
        self.hilo_prediccion.start()

    def guardar_resultados(self):
        if self.archivo_seleccionado is None:
            print("⚠️ No hay imagen cargada para guardar resultados.")
            return
        carpeta = filedialog.askdirectory(title="Seleccionar carpeta para guardar los resultados")
        if not carpeta:
            return
        nombre_base = os.path.splitext(os.path.basename(self.archivo_seleccionado))[0]
        for etiqueta, sufijo in zip([self.etiqueta_mascara, self.etiqueta_esqueleto, self.etiqueta_longitud],
            ["_segmentacion.png", "_esqueleto.png", "_longitud.png"]):
            if hasattr(etiqueta, "image") and etiqueta.image:
                try:
                    imagen_pil = etiqueta.image._light_image
                    ruta_guardado = os.path.join(carpeta, nombre_base + sufijo)
                    imagen_pil.save(ruta_guardado)
                    print(f"💾 Guardado: {ruta_guardado}")
                except Exception as e:
                    print(f"❌ Error al guardar {sufijo}: {e}")
        print("✅ Imágenes de resultados guardadas correctamente.")

    def realizar_prediccion(self):
        print("🧠 EJECUCIÓN INICIADA…")
        ruta = self.archivo_seleccionado
        nombre_imagen = os.path.splitext(os.path.basename(ruta))[0] + ".png"
        print (nombre_imagen)
        
        REALIZAR_PREDICCION.predecir_con_mejor_modelo("best.pt", ruta)
        REALIZAR_PREDICCION.guardar_predicciones_postprocesadas()

        directorio = os.path.dirname(ruta)
        REALIZAR_PREDICCION.postprocesado("runs/segment/predict/labels", directorio, "FINAL")

        mascara = REALIZAR_PREDICCION.procesar_imagen(os.path.join("FINAL", nombre_imagen), "MASCARAS_FINAL")
        self.after(0, lambda: self.mostrar_imagen(mascara, self.etiqueta_mascara))

        esqueleto = REALIZAR_PREDICCION.obtener_imagenes_skeleton("MASCARAS_FINAL", "SKELETON")
        self.after(0, lambda: self.mostrar_imagen(esqueleto, self.etiqueta_esqueleto))

        pixeles_blancos = np.all(esqueleto == [255, 255, 255], axis=-1)
        resultado = np.zeros_like(esqueleto)
        resultado[pixeles_blancos] = [255, 255, 255]
        self.after(0, lambda: self.mostrar_imagen(resultado, self.etiqueta_longitud))

        total_pixeles = np.sum(pixeles_blancos)
        print(f"🧪 Píxeles blancos detectados: {total_pixeles}")
        self.after(0, lambda: self.etiqueta_resultado1.configure(
            text=f"📐 Píxeles totales: {total_pixeles} px"
        ))

        #0,0258 mm en anchura y 0,0256 en longitud
        longitud = total_pixeles * 0.0256
        self.after(0, lambda: self.etiqueta_resultado2.configure(
            text=f"📐 Longitud estimada: {longitud} mm"
        ))

        REALIZAR_PREDICCION.eliminar_carpetas()


if __name__ == "__main__":
    app = Aplicacion()
    app.mainloop()


📁 Archivo cargado: C:/Users/urkoa/OneDrive - Universidad de Burgos/TFG/Radiografias-ESTUDIO LONGITUD DE TRABAJO-IA/D5 24mm/1.jpg
🧠 EJECUCIÓN INICIADA…
1.png

image 1/1 C:\Users\urkoa\OneDrive - Universidad de Burgos\TFG\Radiografias-ESTUDIO LONGITUD DE TRABAJO-IA\D5 24mm\1.jpg: 1024x800 1 ROOT, 1 TOOTH, 193.4ms
Speed: 8.9ms preprocess, 193.4ms inference, 5.4ms postprocess per image at shape (1, 3, 1024, 800)
Results saved to runs\segment\predict
1 label saved to runs\segment\predict\labels
🧪 Píxeles blancos detectados: 975
✅ Carpeta eliminada: predicciones_postprocesadas
✅ Carpeta eliminada: runs
✅ Carpeta eliminada: FINAL
✅ Carpeta eliminada: MASCARAS_FINAL
✅ Carpeta eliminada: SKELETON
📁 Archivo cargado: C:/Users/urkoa/OneDrive - Universidad de Burgos/TFG/Radiografias-ESTUDIO LONGITUD DE TRABAJO-IA/D5 24mm/2.jpg
🧠 EJECUCIÓN INICIADA…
2.png

image 1/1 C:\Users\urkoa\OneDrive - Universidad de Burgos\TFG\Radiografias-ESTUDIO LONGITUD DE TRABAJO-IA\D5 24mm\2.jpg: 1024x800 1 ROOT, 1 TOOTH